In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

import numpy as np

In [2]:
from  metal.utils import hard_to_soft
import torch

N = 1200
X = np.random.random((N,500)) * 2 - 1

Y = np.zeros((N,1))
Y[:,0] = (X[:,0] > X[:,1] + 0.5).astype(int) + 1

X = torch.tensor(X, dtype=torch.float)
Y = torch.tensor(Y, dtype=torch.long)

X_train = X[:1000]
X_dev   = X[1000:1100]
X_test  = X[1100:]

Y_train = Y[:1000, 0]
Y_dev   = Y[1000:1100]
Y_test  = Y[1100:, 0]

In [3]:
from metal.end_model import EndModel
from metal.end_model.em_defaults import *

em = EndModel(
    seed=1,
    dropout=0.0,
    batchnorm=False,
    layer_out_dims=[500,4,2],
)

print(em.config["train_config"].keys())

em.train(X_train, Y_train, X_dev, Y_dev, n_epochs=40, print_every=5)
em.score(X_test, Y_test)


Network architecture:
Sequential(
  (0): IdentityModule()
  (1): Sequential(
    (0): Linear(in_features=500, out_features=4, bias=True)
    (1): ReLU()
  )
  (2): Linear(in_features=4, out_features=2, bias=True)
)

dict_keys(['print_every', 'use_cuda', 'data_loader_config', 'n_epochs', 'l2', 'validation_metric', 'validation_freq', 'optimizer_config', 'scheduler_config', 'checkpoint', 'checkpoint_config'])
dict_keys(['print_every', 'use_cuda', 'data_loader_config', 'n_epochs', 'l2', 'validation_metric', 'validation_freq', 'optimizer_config', 'scheduler_config', 'checkpoint', 'checkpoint_config'])
Saving model at iteration 0 with best score 0.660
[E:0]	Train Loss: 0.704	Dev score: 0.660
Saving model at iteration 1 with best score 0.710
Saving model at iteration 2 with best score 0.740
Saving model at iteration 3 with best score 0.760
[E:5]	Train Loss: 0.049	Dev score: 0.760
Saving model at iteration 9 with best score 0.770
Saving model at iteration 10 with best score 0.780
[E:10]	Train

0.76

In [4]:
rs_max_search = 10

search_space = {
    'n_epochs': [50, 100, 150],
    'batchnorm' : [True, False],
    'dropout': [0, .1, .2, .3, .4, .5],
    'lr': {'range': [1e-5, 1], 'scale': 'log'},
    'layer_out_dims' : [[500,4,2], [500, 50, 2], [500, 100, 2], [500, 200, 2], [500, 400, 2]],
    'print_every': 50
}

In [5]:
# First run basic grid search / random search
from metal.label_model import LabelModel
from metal.end_model import EndModel
from metal.tuners.random_tuner import RandomSearchTuner

def run_random_search(seed=123):
    tuner = RandomSearchTuner(EndModel, seed=seed)
    init_args = []
    train_args = [X_train, Y_train]
    model = tuner.search(init_args, train_args, X_dev, Y_dev, search_space,
                         max_search=rs_max_search, metric='f1',
                        verbose=False)
    return tuner.get_run_stats()

In [6]:

# Hyperband
from metal.label_model import LabelModel
from metal.end_model import EndModel
from metal.tuners.hyperband_tuner import HyperbandTuner

def run_hyperband(seed=123):    
    tuner = HyperbandTuner(EndModel, hyperband_epochs_budget=1000, seed=seed)
    init_args = []
    train_args = [X_train, Y_train]

    model = tuner.search(init_args, train_args, X_dev, Y_dev, search_space, verbose=False)
    return tuner.get_run_stats()

In [ ]:
#%%capture
seeds = [123, 234, 456, 567, 678]
#seeds = [123]
runstats_hyperband = []
runstats_rs = []
for seed in seeds:
    runstats_rs.append(run_random_search(seed=seed))
    runstats_hyperband.append(run_hyperband(seed=seed))
    
print(runstats_hyperband)
print(runstats_rs)


Network architecture:
Sequential(
  (0): IdentityModule()
  (1): Sequential(
    (0): Linear(in_features=500, out_features=4, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3)
  )
  (2): Linear(in_features=4, out_features=2, bias=True)
)

dict_keys(['print_every', 'use_cuda', 'data_loader_config', 'n_epochs', 'l2', 'validation_metric', 'validation_freq', 'optimizer_config', 'scheduler_config', 'checkpoint', 'checkpoint_config'])

Network architecture:
Sequential(
  (0): IdentityModule()
  (1): Sequential(
    (0): Linear(in_features=500, out_features=4, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5)
  )
  (2): Linear(in_features=4, out_features=2, bias=True)
)

dict_keys(['print_every', 'use_cuda', 'data_loader_config', 'n_epochs', 'l2', 'validation_metric', 'validation_freq', 'optimizer_config', 'scheduler_config', 'checkpoint', 'checkpoint_config'])

Network architecture:
Sequential(
  (0): 

In [ ]:
%matplotlib inline

# Plot random search vs hyperband run stats
import matplotlib.pyplot as plt
import matplotlib as mpl

print(runstats_hyperband)

xs_hyperband = [[x["time_elapsed"] for x in z] for z in runstats_hyperband]
ys_hyperband = [[x["best_score"] for x in z] for z in runstats_hyperband]
xs_rs = [[x["time_elapsed"] for x in z] for z in runstats_rs]
ys_rs = [[x["best_score"] for x in z] for z in runstats_rs]

# Extract min and max times for segmented times
flat_xs_hyperband = [item for sublist in xs_hyperband for item in sublist]
flat_xs_rs = [item for sublist in xs_rs for item in sublist]
min_time, max_time = 0, max(flat_xs_hyperband + flat_xs_rs)


# Get list of best scores at time
def get_best_scores_at_time(all_runstats, target_time):
    best_scores = []
    for runstats in all_runstats:
        best_score = 0        
        times = [x["time_elapsed"] for x in runstats]
        scores = [x["best_score"] for x in runstats]
        for time, score in zip(times, scores):
            if time < target_time:
                best_score = score
        best_scores.append(best_score)
    return best_scores
        
# Extract best scores per segment
segments = list(np.arange(0, max_time, 1))
hyperband_scores = [get_best_scores_at_time(runstats_hyperband, t) for t in segments]
rs_scores = [get_best_scores_at_time(runstats_rs, t) for t in segments]

# Extract means and error bars
mean_hyperband_scores = [np.mean(x) for x in hyperband_scores]
mean_rs_scores = [np.mean(x) for x in rs_scores]
#hyperband_stds = [np.std(x) for x in hyperband_scores]
#rs_stds = [np.std(x) for x in rs_scores]
hyperband_stds = [0 for x in hyperband_scores]
rs_stds = [0 for x in rs_scores]

#plt.plot(segments, mean_hyperband_scores, label="Hyperband", marker="o")
#plt.plot(segments, mean_rs_scores, label="Random Search", marker="o")
width = 12
height = 12
plt.figure(figsize=(width, height))
plt.errorbar(segments, mean_hyperband_scores, hyperband_stds, label="Hyperband", marker="o")
plt.errorbar(segments, mean_rs_scores, rs_stds, label="Random Search", marker="o")
plt.legend(loc="best")
plt.title("Best Acc Score Achieved vs Time")
plt.xlabel("Time (s)")
plt.ylabel("Mean Acc")

In [ ]:
%matplotlib inline

# Plot random search vs hyperband run stats
import matplotlib.pyplot as plt
import matplotlib as mpl

print(runstats_hyperband)

xs_hyperband = [[x["time_elapsed"] for x in z] for z in runstats_hyperband]
ys_hyperband = [[x["best_score"] for x in z] for z in runstats_hyperband]
xs_rs = [[x["time_elapsed"] for x in z] for z in runstats_rs]
ys_rs = [[x["best_score"] for x in z] for z in runstats_rs]

# Extract min and max times for segmented times
flat_xs_hyperband = [item for sublist in xs_hyperband for item in sublist]
flat_xs_rs = [item for sublist in xs_rs for item in sublist]
min_time, max_time = 0, max(flat_xs_hyperband + flat_xs_rs)


# Get list of best scores at time
def get_best_scores_at_time(all_runstats, target_time):
    best_scores = []
    for runstats in all_runstats:
        best_score = 0        
        times = [x["time_elapsed"] for x in runstats]
        scores = [x["best_score"] for x in runstats]
        for time, score in zip(times, scores):
            if time < target_time:
                best_score = score
        best_scores.append(best_score)
    return best_scores
        
# Extract best scores per segment
segments = list(np.arange(0, max_time, 1))
hyperband_scores = [get_best_scores_at_time(runstats_hyperband, t) for t in segments]
rs_scores = [get_best_scores_at_time(runstats_rs, t) for t in segments]

# Extract means and error bars
mean_hyperband_scores = [np.mean(x) for x in hyperband_scores]
mean_rs_scores = [np.mean(x) for x in rs_scores]
hyperband_stds = [[-np.percentile(x, 10)+np.mean(x) for x in hyperband_scores], [np.percentile(x, 90)-np.mean(x) for x in hyperband_scores]]
rs_stds = [[-np.percentile(x, 10)+np.mean(x) for x in rs_scores], [np.percentile(x, 90)-np.mean(x) for x in rs_scores]]


#plt.plot(segments, mean_hyperband_scores, label="Hyperband", marker="o")
#plt.plot(segments, mean_rs_scores, label="Random Search", marker="o")
width = 12
height = 12
plt.figure(figsize=(width, height))
plt.errorbar(segments, mean_hyperband_scores, hyperband_stds, label="Hyperband", marker="o")
plt.errorbar(segments, mean_rs_scores, rs_stds, label="Random Search", marker="o")
plt.legend(loc="best")
plt.title("Best Acc Score Achieved vs Time")
plt.xlabel("Time (s)")
plt.ylabel("Mean Acc")

# 